In [2]:
import pandas as pd
import numpy as np
import os

os.listdir('../input')

['new_submission_baseline.csv',
 'df_ref.csv',
 'train.csv',
 'df_ref2.csv',
 'submission_popular.csv',
 'item_metadata.csv',
 'df_full_except_noinfo.csv',
 'test.csv',
 'new_submission.csv',
 'dwell_df.csv',
 'baseline_sorting.csv',
 'dwell_time_agg.csv',
 'test2.csv']

In [3]:
path = '../input/'
df = pd.read_csv(path + 'df_full_except_noinfo.csv')

In [4]:
def only_number(string):
    """reference에서 숫자인 것만 뽑아 int type으로 변환"""
    try:
        int(string)
        return string
    except ValueError:
        return np.nan
    
df.reference = df.reference.apply(only_number)

In [44]:
df.reset_index(drop = True, inplace = True)

In [40]:
ref_list = list(df.reference.values)
first_index = [i for i, r in enumerate(ref_list) if ref_list[i-1] != ref_list[i]]

In [57]:
dwell_df = pd.DataFrame(columns = ['user_id','session_id','reference','dwell_time'])
dwell_df['user_id'] = df.loc[first_index, 'user_id']
dwell_df['session_id'] = df.loc[first_index, 'session_id']
dwell_df['reference'] = df.loc[first_index, 'reference']
dwell_df.reset_index(drop = True, inplace = True)

first_time = df.loc[first_index, 'timestamp']
first_time = pd.to_datetime(first_time, unit = 's')
first_time.reset_index(drop = True, inplace = True)

last_time = pd.Series(np.roll(first_time,-1))

dwell_df['dwell_time'] = (last_time - first_time).astype('timedelta64[s]')

In [58]:
dwell_df.head()

,user_id,session_id,reference,dwell_time
0,00RL8Z82B2Z1,aff3928535f48,NaN,62.0
1,00RL8Z82B2Z1,aff3928535f48,666856,10.0
2,00RL8Z82B2Z1,aff3928535f48,109038,0.0
3,00RL8Z82B2Z1,aff3928535f48,666856,0.0
4,00RL8Z82B2Z1,aff3928535f48,109038,0.0


In [59]:
dwell_df.dropna(inplace = True)
dwell_df = dwell_df[dwell_df.dwell_time>0]
dwell_df.shape

(2249506, 4)

# sort by dwell time

In [60]:
new_submission = pd.read_csv(path + 'new_submission.csv')
new_submission['impressions'] = new_submission['impressions'].apply(lambda x : x.split('|'))
new_submission.head(3)

,user_id,session_id,timestamp,step,reference,impressions
0,004A07DM0IDW,1d688ec168932,1541555792,6,3241426,"[2059240, 2033381, 1724779, 127131, 399441, 10..."
1,00Y1Z24X8084,26b6d294d66e7,1541651766,1,3843244,"[2714480, 4476010, 3843244, 3833012, 9017890, ..."
2,01V3WDTDM5CU,07628a0f5be0b,1541575516,4,2817590,"[3565720, 2947584, 4115018, 2039671, 3836538, ..."


In [63]:
dwell_time_sort = dwell_df.groupby(['user_id', 'session_id'])['reference'].apply(lambda x : ' '.join(x)).to_frame().reset_index()
dwell_time_sort = dwell_time_sort[(dwell_time_sort.user_id.isin(new_submission.user_id)) & (dwell_time_sort.session_id.isin(new_submission.session_id))]
dwell_time_sort.head()

,user_id,session_id,reference
5,0008BO33KUQ0,2d0e2102ee0dc,452021 507861
9,000GO9NY6P4M,c712ede188d00,1618677
11,000IRHJS2DL9,f6ffffd20d43d,346166 4552802 2547840
15,000VBY1D6BP8,033fddaaa99af,5592656
23,0013L641G5P2,05a5589dd7b92,54640


In [64]:
dwell_time_sort.reference = dwell_time_sort.reference.apply(lambda x : x.split(' '))
dwell_time_sort.rename(columns = {'reference' : 'dwell_time_sort_sorted'}, inplace = True)
dwell_time_sort.head(3)

,user_id,session_id,dwell_time_sort_sorted
5,0008BO33KUQ0,2d0e2102ee0dc,"[452021, 507861]"
9,000GO9NY6P4M,c712ede188d00,[1618677]
11,000IRHJS2DL9,f6ffffd20d43d,"[346166, 4552802, 2547840]"


In [67]:
new_submission = pd.merge(new_submission, dwell_time_sort, on = ['user_id', 'session_id'], how = 'left')

In [68]:
def sort_dwell(imp, ref) :
    my = []
    for rr in ref :
        if rr in imp :
            my.append(rr)
            imp.remove(rr)
    imp = my + imp
    return imp

In [69]:
#apply 
apply_index = new_submission[(new_submission.user_id.isin(dwell_time_sort.user_id)) & (new_submission.session_id.isin(dwell_time_sort.session_id))].index
new_submission.loc[apply_index, 'impressions'] = new_submission.loc[apply_index].apply(lambda x : sort_dwell(x['impressions'], x['dwell_time_sort_sorted']), axis = 1)

In [70]:
def mrr(answer, imp) :
    myscore = 0
    
    for i,j in zip(answer, imp) :
        if i in j :
            myscore += 1/(j.index(i) + 1)
    print(myscore / len(imp))

In [71]:
new_submission.reference = new_submission.reference.apply(str)
mrr(new_submission['reference'], new_submission['impressions'])

0.6297955013800194


In [72]:
new_submission.head(3)

,user_id,session_id,timestamp,step,reference,impressions,dwell_time_sort_sorted
0,004A07DM0IDW,1d688ec168932,1541555792,6,3241426,"[2059240, 1050068, 3241426, 2033381, 1724779, ...","[2059240, 1050068, 3241426]"
1,00Y1Z24X8084,26b6d294d66e7,1541651766,1,3843244,"[2714480, 4476010, 3843244, 3833012, 9017890, ...",NaN
2,01V3WDTDM5CU,07628a0f5be0b,1541575516,4,2817590,"[2817590, 3565720, 2947584, 4115018, 2039671, ...",[2817590]


In [73]:
new_submission.impressions = new_submission.impressions.apply(lambda x : ' '.join(x))
ind = new_submission.loc[~new_submission.dwell_time_sort_sorted.isnull()].index
new_submission.loc[ind, 'dwell_time_sort_sorted'] = new_submission.loc[ind]['dwell_time_sort_sorted'].apply(lambda x : ' '.join(x))
new_submission.head(3)

,user_id,session_id,timestamp,step,reference,impressions,dwell_time_sort_sorted
0,004A07DM0IDW,1d688ec168932,1541555792,6,3241426,2059240 1050068 3241426 2033381 1724779 127131...,2059240 1050068 3241426
1,00Y1Z24X8084,26b6d294d66e7,1541651766,1,3843244,2714480 4476010 3843244 3833012 9017890 198100...,NaN
2,01V3WDTDM5CU,07628a0f5be0b,1541575516,4,2817590,2817590 3565720 2947584 4115018 2039671 383653...,2817590


In [74]:
#new_submission.to_csv(path + 'new_submission_dwell_time.csv', index = False)

# baseline(ref count) + dwell_time

In [76]:
baseline = pd.read_csv(path + 'new_submission_baseline.csv')
baseline.head(3)

,user_id,session_id,timestamp,step,reference,impressions,ref_count_sorted
0,004A07DM0IDW,1d688ec168932,1541555792,6,3241426,1050068 2059240 3241426 2033381 1724779 127131...,1050068 2059240 3241426
1,00Y1Z24X8084,26b6d294d66e7,1541651766,1,3843244,2714480 4476010 3843244 3833012 9017890 198100...,NaN
2,01V3WDTDM5CU,07628a0f5be0b,1541575516,4,2817590,2817590 3565720 2947584 4115018 2039671 383653...,2817590


In [79]:
dwell_sort = pd.read_csv(path + 'new_submission_dwell_time.csv')
dwell_sort.head(3)

,user_id,session_id,timestamp,step,reference,impressions,dwell_time_sort_sorted
0,004A07DM0IDW,1d688ec168932,1541555792,6,3241426,2059240 1050068 3241426 2033381 1724779 127131...,2059240 1050068 3241426
1,00Y1Z24X8084,26b6d294d66e7,1541651766,1,3843244,2714480 4476010 3843244 3833012 9017890 198100...,NaN
2,01V3WDTDM5CU,07628a0f5be0b,1541575516,4,2817590,2817590 3565720 2947584 4115018 2039671 383653...,2817590


In [82]:
baseline_add = pd.merge(baseline, dwell_sort.drop('impressions', axis = 1))
del dwell_sort, baseline

In [84]:
baseline_add.head(10)

,user_id,session_id,timestamp,step,reference,impressions,ref_count_sorted,dwell_time_sort_sorted
0,004A07DM0IDW,1d688ec168932,1541555792,6,3241426,1050068 2059240 3241426 2033381 1724779 127131...,1050068 2059240 3241426,2059240 1050068 3241426
1,00Y1Z24X8084,26b6d294d66e7,1541651766,1,3843244,2714480 4476010 3843244 3833012 9017890 198100...,NaN,NaN
2,01V3WDTDM5CU,07628a0f5be0b,1541575516,4,2817590,2817590 3565720 2947584 4115018 2039671 383653...,2817590,2817590
3,02AOAVF9PVYH,4a01c3afbc224,1541681239,45,1177554,7304020 1177554 1451247 559056 1045096 1963879...,749491 552311 7304020 1177554 1177542 6639820 ...,552311 749491 7304020 6639820 1177542 552311 6...
4,0339C84S24ET,3e7d55eae386f,1541622146,3,8511456,8511456 10598152 7031496 7348404 5753730 72388...,4957186 8511456,8511456
5,0386OH8JDE1Q,e09591d07cdef,1541620433,1,5488246,1193320 5488246 3858774 4552034 10620372 22696...,NaN,NaN
6,03LTH89QY623,7663406cf586c,1541554127,3,59566,59566 241961 906477 991561 353701 1149665 7725...,358241 59566,358241 59566
7,03XH0JWCWHAM,73f4c417ff730,1541565912,48,43133,6298428 43133 1363450 1727149 42913 5970348 25...,42692 42858 6298428 3148690 968925 42618 64520...,968925 42692 5762742 43133 42692 3148690 62984...
8,062OG5L9831R,289b34a2bb4dd,1541663095,20,9914686,9914686 8474478 5828606 6618430 7828086 922547...,9914686 8474478,8474478 9914686
9,06OT6WXN246O,ca92389e5ea65,1541688359,13,9984412,5480450 5892026 9984412 4667086 3152528 434128...,5480450 5892026 9984412,5480450 5892026
